# Documentation  

Anaconda 2019.07 버전의 python 3.7.4 defalut 값으로 진행  
  
Python = 3.7.4  
Pandas = 0.25.1   
Numpy = 1.16.5  
urllib3 = 1.24.2  
BeautifulSoup = 4.8.0  
requests = 2.22.0   

In [1]:
#import module

import pandas as pd
import numpy as np
import time
import re
import os
import datetime as dt

import urllib3
from bs4 import BeautifulSoup
import requests

import selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

import warnings
warnings.filterwarnings(action='ignore')


### 다른 사이트의 경우 IT이외의 다른 분야의 기사가 많이 혼재하여 선제적으로 IT관련 뉴스가 가장 많은
### 네이버 뉴스 IT/과학 헤드라인
### ITWorld korea의 머신러닝, 클라우드, 빅데이터 파트
### ZDnet의 인공지능 파트에서 데이터를 추출할 예정입니다

#### 현재는 네이버 뉴스의 IT/과학 헤드라인을 먼저 실행중이며, 하나의 URL에서 전체 뉴스의 URL로 확장하는 작업중에 있습니다  
#### IT/과학 헤드라인의 헤드라인 더보기 전까지 기사의 제목과 URL을 추출후 DataFrame안에 저장하여 기록하고  
#### 이후 저장한 URL에 개별 접속하여 기사의 본문을 가져와서 저장하는 형식으로 진행하였습니다

In [58]:
news = 'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105' ## IT/과학 헤드라인 뉴스  url

res = requests.get(news, headers={'User-Agent':'Chrome/85.0.4183.121'},verify = False) 
# requests이용하여 뉴스 주소 Load, 네이버 크롬 차단 방지 위해 chrome version을 입력하고 뉴스 주소를 설정
# SSL ERROR 방지를 위해 Verify False 설정
soup = BeautifulSoup(res.text, "html.parser") # html기준으로 parsing

title_list =[] # 제목을 저장하기 위한 list 생성
url_list = [] #URL을 저장하기 위한 list 생성

results = soup.select("#main_content a")  # 기사 제목과 URL이 모두 html main_content a에 저장되어 있어서 해당 부분을 select하여 추출


# 제목, 링크 출력 ---------------------------(2)
for result in results:
    if result.string == None: # 값이 존재하지 않을 경우 해당 루프 건너 뜀
        continue
    
    if result.attrs['href'] in ['#'] : # 헤드라인 더보기의 end code가 #이므로 #일 경우 loop문을 Break하여 종료함
        break
    
    if result.attrs['href'].startswith('http') == False : 
        # 해당 href에서 url 코드가 http로 시작하지 않을경우 url이 아니라고 판단하여 해당 부분 skip
        continue
    

    title_list.append(str.strip(result.string)) #기록된 기사 제목을 리스트에 추가
    url_list.append(result.attrs['href']) #기록된 기사 URL을 리스트에 추가


df = pd.DataFrame({'title': title_list,'url':url_list}) # 기사제목과, URL을 dataframe에 임시 저장
df['title'] = df.title.str.replace('\\',"") # 따옴표가  &apos 형태로 기록되어 해당 문자를 따옴표로 대체
df.head(10)

,title,url
0,"네이버쇼핑 2% 수수료 논란…네이버 ""검색노출 대가, 판매 수수료 아니다""",https://news.naver.com/main/read.nhn?mode=LSD&...
1,"네이버 ""스마트스토어 수수료 업계 최저…네이버쇼핑 입점 필수 아냐""",https://news.naver.com/main/read.nhn?mode=LSD&...
2,"네이버 ""스마트스토어 고액 수수료·고정비, 사실 아냐""",https://news.naver.com/main/read.nhn?mode=LSD&...
3,"네이버 ""스마트스토어 수수료 오해 마세요…업계 최저 맞아요""",https://news.naver.com/main/read.nhn?mode=LSD&...
4,"전세계 방송인들, 12일부터 닷새간 한류에 `흠뻑`",https://news.naver.com/main/read.nhn?mode=LSD&...
5,'한류'에 빠지다..‘밉콤(MIPCOM)’ 주빈국으로 개최,https://news.naver.com/main/read.nhn?mode=LSD&...
6,"한국 홀대하던 애플, 처음으로 ‘아이폰12’ 1차 출시국에 포함할 듯",https://news.naver.com/main/read.nhn?mode=LSD&...
7,고화질 영상 삼성 vs 첫 5G폰 애플…'10월 대전' 승자는,https://news.naver.com/main/read.nhn?mode=LSD&...
8,[잇써보니]'유부남 로망에 최적화된 기기'...5GX클라우드 게임,https://news.naver.com/main/read.nhn?mode=LSD&...
9,세계시장 휘젓는 中게임···안방에 갇힌 韓게임,https://news.naver.com/main/read.nhn?mode=LSD&...


### 중복된 기사를 제거하기 위해 헤드라인 뉴스 밑에 있는 중복된 기사를 제거하는 작업이 필요하다

이를 위해 해당 중복된 뉴스의 tag를 찾아 제목을 기록하고 이와 똑같은 뉴스를 찾아 제거한다

In [71]:
# 중복된 기사 list 추출
duplicate_list = []
dup_title = soup.find_all("li",{"class":"cluster_item as_line"})
for dup in dup_title:
    duplicate_list.append(dup.a.get_text())
duplicate_list

['네이버 "스마트스토어 수수료 업계 최저…네이버쇼핑 입점 필수 아냐"',
 '네이버 "스마트스토어 고액 수수료·고정비, 사실 아냐"',
 '네이버 "스마트스토어 수수료 오해 마세요…업계 최저 맞아요"',
 '[사이언스카페] 노벨상 유전자 가위 기술로 코로나 단 5분에 확인한다',
 '[사이언스카페] 수박에 소금을 뿌리면 왜 더 달까',
 '[사이언스카페] "아마존 물류창고… 로봇 도입 이후 인적사고 33% 증가"',
 '"8년 기다렸다" \'사이버펑크 2077\'에 설레는 PC 업계[오지현의 하드캐리]',
 '모바일 게임이 대세? 2020년은 콘솔 부활의 해![오지현의 하드캐리]',
 "운영자가 최대 리스크? '슈퍼계정' 사태로 돌아본 GM 흑역사[오지현의 하드캐리]"]

In [72]:
df

,title,url
0,"네이버쇼핑 2% 수수료 논란…네이버 ""검색노출 대가, 판매 수수료 아니다""",https://news.naver.com/main/read.nhn?mode=LSD&...
1,"네이버 ""스마트스토어 수수료 업계 최저…네이버쇼핑 입점 필수 아냐""",https://news.naver.com/main/read.nhn?mode=LSD&...
2,"네이버 ""스마트스토어 고액 수수료·고정비, 사실 아냐""",https://news.naver.com/main/read.nhn?mode=LSD&...
3,"네이버 ""스마트스토어 수수료 오해 마세요…업계 최저 맞아요""",https://news.naver.com/main/read.nhn?mode=LSD&...
4,"전세계 방송인들, 12일부터 닷새간 한류에 `흠뻑`",https://news.naver.com/main/read.nhn?mode=LSD&...
5,'한류'에 빠지다..‘밉콤(MIPCOM)’ 주빈국으로 개최,https://news.naver.com/main/read.nhn?mode=LSD&...
6,"한국 홀대하던 애플, 처음으로 ‘아이폰12’ 1차 출시국에 포함할 듯",https://news.naver.com/main/read.nhn?mode=LSD&...
7,고화질 영상 삼성 vs 첫 5G폰 애플…'10월 대전' 승자는,https://news.naver.com/main/read.nhn?mode=LSD&...
8,[잇써보니]'유부남 로망에 최적화된 기기'...5GX클라우드 게임,https://news.naver.com/main/read.nhn?mode=LSD&...
9,세계시장 휘젓는 中게임···안방에 갇힌 韓게임,https://news.naver.com/main/read.nhn?mode=LSD&...


In [74]:
# 중복된 기사 제거
title_row = []
for row, title in enumerate(df['title']) :
    if title in test_list :
        title_row.append(row)

df.drop(title_row,inplace=True)

#### 위에 기사 제목과 비교하면 중복된 기사인 1,2,3 번의 뉴스가 제거된 것을 볼 수 있다

In [75]:
df

,title,url
0,"네이버쇼핑 2% 수수료 논란…네이버 ""검색노출 대가, 판매 수수료 아니다""",https://news.naver.com/main/read.nhn?mode=LSD&...
4,"전세계 방송인들, 12일부터 닷새간 한류에 `흠뻑`",https://news.naver.com/main/read.nhn?mode=LSD&...
5,'한류'에 빠지다..‘밉콤(MIPCOM)’ 주빈국으로 개최,https://news.naver.com/main/read.nhn?mode=LSD&...
6,"한국 홀대하던 애플, 처음으로 ‘아이폰12’ 1차 출시국에 포함할 듯",https://news.naver.com/main/read.nhn?mode=LSD&...
7,고화질 영상 삼성 vs 첫 5G폰 애플…'10월 대전' 승자는,https://news.naver.com/main/read.nhn?mode=LSD&...
8,[잇써보니]'유부남 로망에 최적화된 기기'...5GX클라우드 게임,https://news.naver.com/main/read.nhn?mode=LSD&...
9,세계시장 휘젓는 中게임···안방에 갇힌 韓게임,https://news.naver.com/main/read.nhn?mode=LSD&...
10,구글 ‘인앱결제’ 의무화 논란… 시장지배력 강화 견제 나선다,https://news.naver.com/main/read.nhn?mode=LSD&...
11,"[김현아의 IT세상읽기]구글과 네이버의 독과점, 같은점과 다른점",https://news.naver.com/main/read.nhn?mode=LSD&...


### 네이버 뉴스는 IT/과학이기 때문에 과학과 관련된 단어 리스트 추가를 통해
### 과학과 관련된 뉴스를 최대한 제거하는 작업이 필요하다고 판단된다

이를 위해 과학에 국한된 단어 리스트를 추가하여 기사 제목이나 본문에 해당 단어 빈도수가 높을 경우 제거

science_list = ['노벨','노벨화학상','화학','과학자'] # 10.08일자 과학 단어 list

샘플 코드로서 저장된 URL중 하나에 접속하여 해당 기사의 제목과 본문 추출


In [3]:
science_list = ['노벨','노벨화학상','화학','과학자'] # 10.08일자 과학 단어 list

샘플 코드로서 저장된 URL중 하나에 접속하여 해당 기사의 제목과 본문 추출

In [60]:
url =' https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=629&aid=0000045669'  ## 각 뉴스 URL 주소 입력
title_list2 =[] # 기사 제목 저장 LIST 생성
news_list2 = [] # 뉴스 본문 저장 LIST 생성


responce = requests.get(url, headers={'User-Agent':'Chrome/85.0.4183.121'},verify = False) 
news = BeautifulSoup(responce.text,'html.parser')
content = news.select("#articleBodyContents") # 기사 본문 저장되어 있는 html주소 복사


news_content = content[0]

for target in news_content.find_all('script'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
    target.extract()

for target in news_content.find_all('a'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
    target.extract()

for target in news_content.find_all('span'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
    target.extract()

for target in news_content.find_all('div'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
    target.extract()
    
for target in news_content.find_all('iframe'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
    target.extract()

    
for target in news_content.find_all('br'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
    target.replace_with("\n")    

    
news_content


<div class="_article_body_contents" id="articleBodyContents">
<!-- 본문 내용 -->
<!-- TV플레이어 -->
<!-- // TV플레이어 -->



<strong>7일 국회 과방위 과기정통부 국정감사서 '구글 수수료 30%' 관련 정부 대책 마련 촉구</strong>

[더팩트│최수진 기자] 국회에서 구글의 수수료 30% 정책에 대한 우려의 목소리가 나왔다. 정부가 적극적으로 나서서 글로벌 IT 공룡의 갑질을 막아야 한다는 주장이다.

7일 국회 과학기술방송통신위원회(과방위)의 과기정통부 국정감사에서 한준호 더불어민주당 의원은 "세계에서 두 번째로 큰 인터넷 시장인 인도에서 150개 스타트업 기업들이 비공식적으로 연합해 구글의 인앱결제 정책을 2022년 4월까지 연기했다"며 "인도에서는 구글이 무릎을 꿇은 것이다. 구글플레이 마켓 규모로 보면 인도 시장보다 국내 시장이 크다. 우리도 정부 차원에서 스타트업들과 함께 인도 사례를 참고해서 구글과 협상해야 한다"고 말했다.

이어 "정부에서 믿음을 줘야 한다"며 "엔씨소프트 같은 경우도 왜 참고인 출석을 안 했겠냐. 구글의 눈치를 보는 것이다. 기업이 눈치를 보지 않도록 정부가 우리 스타트업과 기업들을 보살펴주고 방패가 될 필요가 있다"고 덧붙였다.

앞서 구글은 지난 9월 온라인 간담회를 열고, 내년부터 플레이스토어에 등록된 모든 앱 및 콘텐츠에 일괄적으로 30%의 수수료를 부과한다고 밝혔다. 이와 함께 인앱결제(IAP)도 강제한다. 신규 앱은 내년 1월 20일부터 적용되며, 기존 앱은 내년 10월부터 변경된다.

문제는 구글의 결정을 따르지 않는 업체가 플레이스토어에서 퇴출당할 가능성이 존재한다는 점이다. 이에 국감에서도 관련된 우려를 표명했다.



홍정민 더불어민주당 의원 역시 "보통 규제라고 하면 시장이나 정부를 보는 입장에 따라 찬반이 갈리는데 구글 사안에 대해서는 모두 한목소리를 내고 있다"며 "정부가 나서서 공정거래법, 전기통신사업법 등 적용해

해당 내용을 Text로 전환, 기사와 유사하게 하기 위해 Selenium 사용시 '\n'일 경우 enter를 입력하게 하여 기사를 붙여넣기 할 예정이다


In [61]:
news_text = news_content.get_text()
news_text = news_text.replace("\t", "").replace('\'','') # text로 전환하며 저장되는 기타 \t \'를 각각 빈칸으로 변환
news_text


'\n\n\n\n\n\n\n7일 국회 과방위 과기정통부 국정감사서 구글 수수료 30% 관련 정부 대책 마련 촉구\n\n[더팩트│최수진 기자] 국회에서 구글의 수수료 30% 정책에 대한 우려의 목소리가 나왔다. 정부가 적극적으로 나서서 글로벌 IT 공룡의 갑질을 막아야 한다는 주장이다.\n\n7일 국회 과학기술방송통신위원회(과방위)의 과기정통부 국정감사에서 한준호 더불어민주당 의원은 "세계에서 두 번째로 큰 인터넷 시장인 인도에서 150개 스타트업 기업들이 비공식적으로 연합해 구글의 인앱결제 정책을 2022년 4월까지 연기했다"며 "인도에서는 구글이 무릎을 꿇은 것이다. 구글플레이 마켓 규모로 보면 인도 시장보다 국내 시장이 크다. 우리도 정부 차원에서 스타트업들과 함께 인도 사례를 참고해서 구글과 협상해야 한다"고 말했다.\n\n이어 "정부에서 믿음을 줘야 한다"며 "엔씨소프트 같은 경우도 왜 참고인 출석을 안 했겠냐. 구글의 눈치를 보는 것이다. 기업이 눈치를 보지 않도록 정부가 우리 스타트업과 기업들을 보살펴주고 방패가 될 필요가 있다"고 덧붙였다.\n\n앞서 구글은 지난 9월 온라인 간담회를 열고, 내년부터 플레이스토어에 등록된 모든 앱 및 콘텐츠에 일괄적으로 30%의 수수료를 부과한다고 밝혔다. 이와 함께 인앱결제(IAP)도 강제한다. 신규 앱은 내년 1월 20일부터 적용되며, 기존 앱은 내년 10월부터 변경된다.\n\n문제는 구글의 결정을 따르지 않는 업체가 플레이스토어에서 퇴출당할 가능성이 존재한다는 점이다. 이에 국감에서도 관련된 우려를 표명했다.\n\n\n\n홍정민 더불어민주당 의원 역시 "보통 규제라고 하면 시장이나 정부를 보는 입장에 따라 찬반이 갈리는데 구글 사안에 대해서는 모두 한목소리를 내고 있다"며 "정부가 나서서 공정거래법, 전기통신사업법 등 적용해 규제해주길 바라고 있다. 정부가 개입을 안 하면 스타트업같이 취약한 경우는 앱 생태계 자체가 무너질 수 있는 중대한 사안"이라고 강조했다.\n\n이어 그는 "스타트업을 비롯해 인터넷 콘텐

### Selenium활용하여 Autoway에 자동 로그인

#### RPA 기사와 유사하게 OTP 승인 없이 로그인 할 수 있는 ID를 받아서 Selenium의 chrome driver를 실행 후 해당 모든 기사의 제목과 url을 입력할 수 있도록 할 예정이다

In [ ]:
path = "D:/chromedriver.exe"    # 크롬드라이버 설치 경로 설정

In [ ]:
set_id = "" #id 입력
pwd = "" # pwd 입력

In [ ]:
# Selenium Load
driver = webdriver.Chrome(path)
# site login
driver.get('https://autoway.hyundai.net/')

driver.find_element_by_xpath('//*[@id="form1"]/div[3]/div[2]').click()
driver.find_element_by_xpath('//*[@id="drpGroupCopList"]').click()
driver.find_element_by_xpath('//*[@id="drpGroupCopList"]/option[2]').click()


log = driver.find_element_by_id("userID") # ID 입력하는 format
log.send_keys(set_id) # id입력
time.sleep(3)
pwd = driver.find_element_by_id("password")
pwd.send_keys(pwd)
driver.find_element_by_xpath('//*[@id="Login"]').click()
time.sleep(10)

# test

In [67]:
df_list

['네이버쇼핑 2% 수수료 논란…네이버 "검색노출 대가, 판매 수수료 아니다"',
 '네이버 "스마트스토어 수수료 업계 최저…네이버쇼핑 입점 필수 아냐"',
 '네이버 "스마트스토어 고액 수수료·고정비, 사실 아냐"',
 '네이버 "스마트스토어 수수료 오해 마세요…업계 최저 맞아요"',
 '전세계 방송인들, 12일부터 닷새간 한류에 `흠뻑`',
 "'한류'에 빠지다..‘밉콤(MIPCOM)’ 주빈국으로 개최",
 '한국 홀대하던 애플, 처음으로 ‘아이폰12’ 1차 출시국에 포함할 듯',
 "고화질 영상 삼성 vs 첫 5G폰 애플…'10월 대전' 승자는",
 "[잇써보니]'유부남 로망에 최적화된 기기'...5GX클라우드 게임",
 '세계시장 휘젓는 中게임···안방에 갇힌 韓게임',
 '구글 ‘인앱결제’ 의무화 논란… 시장지배력 강화 견제 나선다',
 '[김현아의 IT세상읽기]구글과 네이버의 독과점, 같은점과 다른점']